In [1]:
import os
print(os.getcwd())
print('files')
print(os.listdir())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/udacityci1/code/Users/odl_user_223037
files
['.amlignore', '.amlignore.amltmp', '.ipynb_aml_checkpoints', 'conda_dependencies.yml', 'train.py', 'udacity-project (1).ipynb', 'udacity-project (1).ipynb.amltmp']


In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-223037
Azure region: southcentralus
Subscription id: b968fb36-f06a-4c76-a15f-afab68ae7667
Resource group: aml-quickstarts-223037


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "proj1cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [16]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--C': uniform(0.1, 1.0), '--max_iter': choice(50,100,200)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

    
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                              hyperparameter_sampling=ps,
                              policy=policy,
                              primary_metric_name="accuracy",
                              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                              max_total_runs=100,
                              max_concurrent_runs=4)

In [17]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_d4c9606c-e1ae-43c2-b6aa-78fc63c094e2
Web View: https://ml.azure.com/runs/HD_d4c9606c-e1ae-43c2-b6aa-78fc63c094e2?wsid=/subscriptions/b968fb36-f06a-4c76-a15f-afab68ae7667/resourcegroups/aml-quickstarts-223037/workspaces/quick-starts-ws-223037&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-01-20T10:34:29.753398][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-01-20T10:34:30.4998990Z][SCHEDULER][INFO]Scheduling job, id='HD_d4c9606c-e1ae-43c2-b6aa-78fc63c094e2_0' 
[2023-01-20T10:34:30.6221974Z][SCHEDULER][INFO]Scheduling job, id='HD_d4c9606c-e1ae-43c2-b6aa-78fc63c094e2_1' 
[2023-01-20T10:34:30.6865041Z][SCHEDULER][INFO]Scheduling job, id='HD_d4c9606c-e1ae-43c2-b6aa-78fc63c094e2_2' 
[2023-01-20T10:34:30.8466084Z][SCHEDULER][INFO]Scheduling job, id='HD_d4c9606c-e1ae-43c2-b6aa-78fc63c094e2_3' 
[2023-01-20T10:34:30.808579][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

In [6]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
final_model = best_run.register_model(model_name = 'hyp_best_run',model_path = 'outputs/model.joblib')

ModelPathNotFoundException: ModelPathNotFoundException:
	Message: Could not locate the provided model_path /outputs/model.joblib in the set of files uploaded to the run: ['azureml-logs/20_image_build_log.txt', 'logs/azureml/dataprep/0/backgroundProcess.log', 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log', 'logs/azureml/dataprep/0/rslex.log.2023-01-20-08', 'system_logs/cs_capability/cs-capability.log', 'system_logs/hosttools_capability/hosttools-capability.log', 'system_logs/lifecycler/execution-wrapper.log', 'system_logs/lifecycler/lifecycler.log', 'system_logs/metrics_capability/metrics-capability.log', 'system_logs/snapshot_capability/snapshot-capability.log', 'user_logs/std_log.txt']
                See https://aka.ms/run-logging for more details.
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Could not locate the provided model_path /outputs/model.joblib in the set of files uploaded to the run: ['azureml-logs/20_image_build_log.txt', 'logs/azur

In [7]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
file_url= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=file_url)

In [10]:
from train import clean_data
import pandas as pd
from azureml.core import Dataset

# Use the clean_data function to clean your data.
X, y = clean_data(ds)
y=pd.DataFrame(y)
df_full=pd.concat([X,y],axis=1)

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df_full,test_size=0.3)
train_data.head()

train_data.to_csv('training_data.csv')

ds1= ws.get_default_datastore()
ds1.upload(src_dir='.', target_path='training_data')

train_ds = Dataset.Tabular.from_delimited_files(path=[(ds1, ('training_data/training_data.csv'))])


Uploading an estimated of 10 files
Target already exists. Skipping upload for training_data/.amlignore
Target already exists. Skipping upload for training_data/.amlignore.amltmp
Target already exists. Skipping upload for training_data/conda_dependencies.yml
Target already exists. Skipping upload for training_data/train.py
Target already exists. Skipping upload for training_data/training_data.csv
Target already exists. Skipping upload for training_data/udacity-project (1).ipynb
Target already exists. Skipping upload for training_data/udacity-project (1).ipynb.amltmp
Target already exists. Skipping upload for training_data/.ipynb_aml_checkpoints/udacity-project (1)-checkpoint2023-0-20-8-45-3Z.ipynb
Target already exists. Skipping upload for training_data/.ipynb_aml_checkpoints/udacity-project (1)-checkpoint2023-0-20-8-6-15Z.ipynb
Target already exists. Skipping upload for training_data/__pycache__/train.cpython-38.pyc
Uploaded 0 files


In [11]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_ds,
    label_column_name='y',
    n_cross_validations=5,
    compute_target = cpu_cluster)

In [12]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run= exp.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on proj1cluster with default configuration
Running on remote compute: proj1cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_bf69886b-7d9a-4c1c-a6a3-e480e07abffe,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

In [13]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
#best_run
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)

import joblib
joblib.dump(fitted_model,'automl_best_model.pkl')

from azureml.core.model import Model

model = Model.register(workspace=ws, model_path="automl_best_model.pkl", model_name="automl-best-model")

Package:azureml-automl-runtime, training version:1.48.0.post1, current version:1.47.0
Package:azureml-core, training version:1.48.0, current version:1.47.0
Package:azureml-dataprep, training version:4.8.3, current version:4.5.7
Package:azureml-dataprep-rslex, training version:2.15.1, current version:2.11.4
Package:azureml-dataset-runtime, training version:1.48.0, current version:1.47.0
Package:azureml-defaults, training version:1.48.0, current version:1.47.0
Package:azureml-interpret, training version:1.48.0, current version:1.47.0
Package:azureml-mlflow, training version:1.48.0, current version:1.47.0
Package:azureml-pipeline-core, training version:1.48.0, current version:1.47.0
Package:azureml-responsibleai, training version:1.48.0, current version:1.47.0
Package:azureml-telemetry, training version:1.48.0, current version:1.47.0
Package:azureml-train-automl-client, training version:1.48.0, current version:1.47.0
Package:azureml-train-automl-runtime, training version:1.48.0, current v

Run(Experiment: udacity-project,
Id: AutoML_bf69886b-7d9a-4c1c-a6a3-e480e07abffe_30,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('26', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False